In [ ]:
from typing import List, Tuple
from math import pi
from datetime import datetime
import rospy
from copy import deepcopy

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
    Vector3,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)
from industrial_reconstruction_msgs.srv import (
    StartReconstruction,
    StartReconstructionRequest,
    StopReconstruction,
    StopReconstructionRequest,
)

from commander.utils import poses_from_yaml, load_scene

CAPTURE = True

rospy.init_node("reconstruction")

load_scene()

plan_goal_srv = rospy.ServiceProxy("commander/plan_goal", PlanGoal)
plan_sequence_srv = rospy.ServiceProxy("commander/plan_sequence", PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy("commander/execute_trajectory", ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy("commander/get_tcp_pose", GetTcpPose)
set_ee_srv = rospy.ServiceProxy("commander/set_ee", SetEe)
pick_place_srv = rospy.ServiceProxy("commander/pick_place", PickPlace)

if CAPTURE:
    start_recon = rospy.ServiceProxy("/start_reconstruction", StartReconstruction)
    stop_recon = rospy.ServiceProxy("/stop_reconstruction", StopReconstruction)


def display_poses(poses: List[Pose], frame_id: str = "base_link") -> None:
    rospy.wait_for_service("/visualize_poses", timeout=10)
    visualize_poses = rospy.ServiceProxy("/visualize_poses", VisualizePoses)
    visualize_poses(frame_id, poses)


def gen_recon_msg(path: str) -> Tuple[StartReconstructionRequest, StopReconstructionRequest]:
    start_srv_req = StartReconstructionRequest()
    start_srv_req.tracking_frame = "rgb_camera_link"
    start_srv_req.relative_frame = "base_link"
    start_srv_req.translation_distance = 0.0
    start_srv_req.rotational_distance = 0.0
    start_srv_req.live = True
    start_srv_req.tsdf_params.voxel_length = 0.02 ##########
    start_srv_req.tsdf_params.sdf_trunc = 0.04 ###########
    start_srv_req.tsdf_params.min_box_values = Vector3(x=0.0, y=0.0, z=0.0)
    start_srv_req.tsdf_params.max_box_values = Vector3(x=0.0, y=0.0, z=0.0)
    start_srv_req.rgbd_params.depth_scale = 1000
    start_srv_req.rgbd_params.depth_trunc = 10000
    start_srv_req.rgbd_params.convert_rgb_to_intensity = False

    stop_srv_req = StopReconstructionRequest()
    path = path + datetime.now().strftime("%m_%d_%H_%M") + ".ply"
    stop_srv_req.mesh_filepath = path

    return start_srv_req, stop_srv_req

In [ ]:
#cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]
cam_home = [-3.062046195720626, -2.042543974749485, -0.9841965193219935, -1.8468536888817202, -1.4911785421602695, -3.2211255818224362]


In [ ]:
# the following args for planner are available:
# - 'ompl' will use the ompl planning pipeline with RTTConnect planner
# - 'ptp' will use the pilz planning pipeline with the PTP planner
# - 'lin' will use the pilz planning pipeline with the LIN planner

plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))

In [ ]:
success = execute_trajectory_srv()

In [ ]:
success = set_ee_srv('rgb_camera_tcp')

In [ ]:
from geometry_msgs.msg import Point, Pose, Quaternion
from tf.transformations import quaternion_from_euler

def pose_from_euler(position, euler_angles):
    # Convert Euler angles to Quaternion
    orientation_quaternion = Quaternion(*quaternion_from_euler(*euler_angles))
    return Pose(position=position, orientation=orientation_quaternion)


# 45º
position0 = Point(0.659, -0.5, 0.171)
euler_angles0 = (3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target0 = pose_from_euler(position0, euler_angles0)

position1 = Point(0.659, -0.25, 0.171)
euler_angles1 = (3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target1 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.659, 0.0, 0.171)
euler_angles1 = (3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target2 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.659, 0.25, 0.171)
euler_angles1 = (3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target3 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.659, 0.50, 0.171)
euler_angles1 = (3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target4 = pose_from_euler(position1, euler_angles1)


# 60º
position0 = Point(0.7, 0.5, 0.203)
euler_angles0 = (3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target5 = pose_from_euler(position0, euler_angles0)

position1 = Point(0.7, 0.25, 0.203)
euler_angles1 = (3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target6 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.7, 0.0, 0.203)
euler_angles1 = (3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target7 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.7, -0.25, 0.203)
euler_angles1 = (3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target8 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.7, -0.50, 0.203)
euler_angles1 = (3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target9 = pose_from_euler(position1, euler_angles1)


#CANTO_1
position1 = Point(0.8, -0.641, 0.246)
euler_angles1 = (3.14, 0.78, 4.71)  # Replace with your desired Euler angles (x, y, z)
target10 = pose_from_euler(position1, euler_angles1)

# 90º
position0 = Point(0.8, -0.5, 0.246)
euler_angles0 = (3.14, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target11 = pose_from_euler(position0, euler_angles0)

position1 = Point(0.8, -0.25, 0.246)
euler_angles1 = (3.14, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target12 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.8, 0.0, 0.246)
euler_angles1 = (3.14, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target13 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.8, 0.25, 0.246)
euler_angles1 = (3.14, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target14 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.8, 0.50, 0.246)
euler_angles1 = (3.14, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target15 = pose_from_euler(position1, euler_angles1)


#CANTO_1
position1 = Point(0.8, 0.641, 0.246)
euler_angles1 = (3.14, -0.78, 4.71)  # Replace with your desired Euler angles (x, y, z)
target16 = pose_from_euler(position1, euler_angles1)


# -60º
position0 = Point(0.9, 0.5, 0.203)
euler_angles0 = (-3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target17 = pose_from_euler(position0, euler_angles0)

position1 = Point(0.9, 0.25, 0.203)
euler_angles1 = (-3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target18 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.9, 0.0, 0.203)
euler_angles1 = (-3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target19 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.9, -0.25, 0.203)
euler_angles1 = (-3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target20 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.9, -0.50, 0.203)
euler_angles1 = (-3.66, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target21 = pose_from_euler(position1, euler_angles1)


# -45º
position0 = Point(0.941, -0.5, 0.171)
euler_angles0 = (-3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target22 = pose_from_euler(position0, euler_angles0)

position1 = Point(0.941, -0.25, 0.171)
euler_angles1 = (-3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target23 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.941, 0.0, 0.171)
euler_angles1 = (-3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target24 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.941, 0.25, 0.171)
euler_angles1 = (-3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target25 = pose_from_euler(position1, euler_angles1)

position1 = Point(0.941, 0.50, 0.171)
euler_angles1 = (-3.92, 0.0, 4.71)  # Replace with your desired Euler angles (x, y, z)
target26 = pose_from_euler(position1, euler_angles1)

# Display Poses
display_poses([target0, target1, target2, target3, target4, target5, 
               target6, target7, target8, target9, target10, target11, 
               target12, target13, target14, target15, target16, target17, target18,
               target19, target20, target21, target22, target23, target24, target25,
               target26])


In [ ]:
if CAPTURE:
    start_recon_req, stop_recon_req = gen_recon_msg("/home/andressaid/capture")

vel = 0.2
acc = 0.2

resp = plan_goal_srv(Goal(pose=target0, vel_scale=0.2, acc_scale=0.2, planner="ptp"))
if resp.success and not resp.configuration_change:
    print("Start recon")
    if CAPTURE:
        start_recon(start_recon_req)
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target1, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target2, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target3, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target4, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target5, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target6, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target7, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target8, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

resp = plan_goal_srv(Goal(pose=target9, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target10, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target11, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target12, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target13, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target14, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target15, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target16, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

resp = plan_goal_srv(Goal(pose=target17, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target18, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target19, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target20, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target21, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target22, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target23, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target24, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target25, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()


resp = plan_goal_srv(Goal(pose=target26, vel_scale=vel, acc_scale=acc, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

if CAPTURE:
    stop_recon(stop_recon_req)